# Initialization
---

### Loading the required Libraries for Time-Series Analysis

In [0]:
%pip install FBProphet

Python interpreter will be restarted.
Collecting FBProphet
 Downloading fbprophet-0.7.1.tar.gz (64 kB)
Requirement already satisfied: Cython>=0.22 in /databricks/python3/lib/python3.8/site-packages (from FBProphet) (0.29.23)
Collecting cmdstanpy==0.9.5
 Downloading cmdstanpy-0.9.5-py3-none-any.whl (37 kB)
Requirement already satisfied: pystan>=2.14 in /databricks/python3/lib/python3.8/site-packages (from FBProphet) (2.19.1.1)
Requirement already satisfied: numpy>=1.15.4 in /databricks/python3/lib/python3.8/site-packages (from FBProphet) (1.20.1)
Requirement already satisfied: pandas>=1.0.4 in /databricks/python3/lib/python3.8/site-packages (from FBProphet) (1.2.4)
Requirement already satisfied: matplotlib>=2.0.0 in /databricks/python3/lib/python3.8/site-packages (from FBProphet) (3.4.2)
Requirement already satisfied: LunarCalendar>=0.0.9 in /databricks/python3/lib/python3.8/site-packages (from FBProphet) (0.0.9)
Requirement already satisfied: convertdate>=2.1.2 in /databricks/python3/lib/python3.8/site-packages (from FBProphet) (2.3.2)
Requirement already satisfied: holidays>=0.10.2 in /databricks/python3/lib/python3.8/site-packages (from FBProphet) (0.12)
Requirement already satisfied: setuptools-git>=1.2 in /databricks/python3/lib/python3.8/site-packages (from FBProphet) (1.2)
Requirement already satisfied: python-dateutil>=2.8.0 in /databricks/python3/lib/python3.8/site-packages (from FBProphet) (2.8.1)
Requirement already satisfied: tqdm>=4.36.1 in /databricks/python3/lib/python3.8/site-packages (from FBProphet) (4.59.0)
Requirement already satisfied: pytz>=2014.10 in /databricks/python3/lib/python3.8/site-packages (from convertdate>=2.1.2->FBProphet) (2020.5)
Requirement already satisfied: pymeeus<=1,>=0.3.13 in /databricks/python3/lib/python3.8/site-packages (from convertdate>=2.1.2->FBProphet) (0.5.11)
Requirement already satisfied: hijri-converter in /databricks/python3/lib/python3.8/site-packages (from holidays>=0.10.2->FBProphet) (2.2.3)
Requirement already satisfied: korean-lunar-calendar in /databricks/python3/lib/python3.8/site-packages (from holidays>=0.10.2->FBProphet) (0.2.1)
Requirement already satisfied: ephem>=3.7.5.3 in /databricks/python3/lib/python3.8/site-packages (from LunarCalendar>=0.0.9->FBProphet) (4.1.3)
Requirement already satisfied: kiwisolver>=1.0.1 in /databricks/python3/lib/python3.8/site-packages (from matplotlib>=2.0.0->FBProphet) (1.3.1)
Requirement already satisfied: cycler>=0.10 in /databricks/python3/lib/python3.8/site-packages (from matplotlib>=2.0.0->FBProphet) (0.10.0)
Requirement already satisfied: pillow>=6.2.0 in /databricks/python3/lib/python3.8/site-packages (from matplotlib>=2.0.0->FBProphet) (8.2.0)
Requirement already satisfied: pyparsing>=2.2.1 in /databricks/python3/lib/python3.8/site-packages (from matplotlib>=2.0.0->FBProphet) (2.4.7)
Requirement already satisfied: six in /databricks/python3/lib/python3.8/site-packages (from cycler>=0.10->matplotlib>=2.0.0->FBProphet) (1.15.0)
Building wheels for collected packages: FBProphet
 Building wheel for FBProphet (setup.py): started
 Building wheel for FBProphet (setup.py): still running...
 Building wheel for FBProphet (setup.py): finished with status 'done'
 Created wheel for FBProphet: filename=fbprophet-0.7.1-py3-none-any.whl size=9534070 sha256=e37ab65c96a794e9d52f7fd0eb753f8b32d1b1babffc3d16f3ba048cbff38445
 Stored in directory: /root/.cache/pip/wheels/d0/d2/ae/c579b7fd160999d35908f3cb8ebcad7ef64ecaca7b78e4c3c8
Successfully built FBProphet
Installing collected packages: cmdstanpy, FBProphet
 Attempting uninstall: cmdstanpy
 Found existing installation: cmdstanpy 0.9.68
 Not uninstalling cmdstanpy at /databricks/python3/lib/python3.8/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-8292a3d8-a483-4911-a095-c775b8dfd947
 Can't uninstall 'cmdstanpy'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

In [0]:
import logging
logging.getLogger('py4j').setLevel(logging.ERROR)

### Loading the data

In [0]:
df1 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/FileStore/shared_uploads/goswamisagard@gmail.com/df1_sample.csv")

In [0]:
df1.printSchema()

root
-- _c0: integer (nullable = true)
-- speed: double (nullable = true)
-- link_id: integer (nullable = true)
-- DateTime: timestamp (nullable = true)
-- hour: integer (nullable = true)
-- minute: integer (nullable = true)
-- weekday: string (nullable = true)
-- year: integer (nullable = true)
-- month: integer (nullable = true)
-- day: integer (nullable = true)
-- day_of_year: integer (nullable = true)

In [0]:
df1 = df1.withColumnRenamed("_c0", "ID")
df1 = df1.withColumnRenamed("DateTime", "ds")
df1 = df1.withColumnRenamed("speed", "y")df1 = df1.withColumnRenamed("_c0", "ID")

In [0]:
from pyspark.sql.types import StringType
df1 = df1.withColumn("link_id",df1.link_id.cast('string'))

In [0]:
df = df1.select('ds', 'link_id', 'y')

In [0]:
from pyspark.sql.functions import count, col

cleaning_filters = (df.ds >= '2018-01-01') & (df.ds < '2022-01-01') & (df.y < 68) & (df.y > 0)

In [0]:
# Filtering rows to clean the data
df = df.filter( cleaning_filters )

In [0]:
df.cache()
df.printSchema()
df.show(10)

root
-- DateTime: timestamp (nullable = true)
-- link_id: string (nullable = true)
-- speed: double (nullable = true)

+-------------------+-------+-----+
 DateTime|link_id|speed|
+-------------------+-------+-----+
2017-12-23 17:22:37|4616241|42.87|
2018-02-20 02:17:03|4456494|13.67|
2021-12-20 09:28:11|4616193|52.19|
2018-12-21 22:18:33|4616225|13.67|
2021-03-31 16:58:03|4456501|13.67|
2019-05-21 03:14:04|4456481|14.29|
2018-09-17 04:58:19|4616192|54.05|
2017-09-02 07:43:22|4362247|49.08|
2019-01-27 21:17:04|4616339|35.41|
2021-06-25 05:38:12|4616323| 1.24|
2020-04-26 08:13:07|4329483|60.27|
2019-01-13 16:48:18|4456452|22.99|
2020-04-01 02:14:03|4616355|59.65|
2018-10-04 02:28:16|4616228|61.51|
2020-02-09 14:08:03|4616305|55.92|
2017-12-20 16:27:28|4616341|10.56|
2021-01-11 16:08:12|4616218|51.57|
2019-05-28 10:54:03|4456502|31.68|
2019-04-06 04:18:12|4616226|27.34|
2021-03-28 12:44:03|4616229|42.25|
+-------------------+-------+-----+
only showing top 20 rows

In [0]:
df.createOrReplaceTempView('data')

# Time-Series Analysis | w.r.t. link_id

In [0]:
# Preparing the input for training model

train = '''
SELECT ds, link_id, AVG(y) AS y
FROM data
GROUP BY ds, link_id
ORDER BY ds, link_id;
'''

df_train = spark.sql(train).toPandas()

## Defining function to train Time-Series Models for each link_id

In [0]:
## Defining function to train Time-Series Models for each link_id

from pyspark.sql.functions import pandas_udf, PandasUDFType

@pandas_udf( df_link_forecast_schema, PandasUDFType.GROUPED_MAP )


def forecast_link( train_data ):
    
    pred_days = 2*365 + 0*30 + 0*7 + 0 # Predicting for 2 years
    
    #removing null values created form the aggregation
    train_data = train_data.dropna()
    
    
    # Training the Time-Series Model
    ts_link_model = Prophet(
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True
    )
    ts_link_model.fit( train_data )
    
    
    # Generating Predictions for the models.
    df_link_future = ts_link_model.make_future_dataframe(
        periods = pred_days * 24 ,
        freq = 'H',
        include_history = True
    )
    
    df_link_forecast = ts_link_model.predict( df_link_future )
    
    
    # Assembling the predicted results
    f_pd = df_link_forecast[ ['ds','yhat', 'yhat_upper', 'yhat_lower'] ].set_index('ds')
    h_pd = train_data[['ds','link_id','y']].set_index('ds')
    
    results_pd = f_pd.join( h_pd, how='left' )
    results_pd.reset_index(level=0, inplace=True)    
    results_pd['link_id'] = train_data['link_id'].iloc[0]
    
    
    # Returning the prediction results
    return results_pd[ ['ds', 'link_id', 'y', 'yhat', 'yhat_upper', 'yhat_lower'] ]  

INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0

## Running the Time-Series Models for individual datasets

In [0]:
from pyspark.sql.functions import current_date

results = (df_link_train
           .groupBy('link_id')
           .apply(forecast_link)
           .withColumn('training_date', current_date() )
)

results.createOrReplaceTempView('link_forecasts')

/databricks/spark/python/pyspark/sql/pandas/group_ops.py:81: UserWarning:

It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.

INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0

## Displaying Results for Predictions

In [0]:
display(results)

ds,link_id,y,yhat,yhat_upper,yhat_lower,training_date
2017-05-08,4329507,34.18,33.058647,44.75394,21.41242,2022-04-24
2017-05-17,4329507,34.18,25.970612,37.704174,13.823777,2022-04-24
2017-06-08,4329507,24.85,28.846054,40.447952,17.597525,2022-04-24
2017-06-08,4329507,16.78,22.64732,34.670284,12.302156,2022-04-24
2017-06-15,4329507,14.91,21.692205,32.6909,10.081255,2022-04-24
2017-06-20,4329507,27.96,22.059807,34.031826,10.509443,2022-04-24
2017-06-22,4329507,27.96,24.059952,35.295666,13.069384,2022-04-24
2017-06-28,4329507,26.72,19.561394,30.781313,7.534235,2022-04-24
2017-07-07,4329507,16.16,21.541931,33.520935,9.812858,2022-04-24
2017-07-15,4329507,26.72,21.76044,33.47502,10.213018,2022-04-24


INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0

In [0]:
results.count()

INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p

In [0]:
%sql
-- Creating a forecast table
create table if not exists forecasts (
  ds timestamp,
  link_id string,
  speed float,
  speed_predicted float,
  speed_predicted_upper float,
  speed_predicted_lower float,
  training_date date
  )
using delta
partitioned by (training_date);

In [0]:
%sql

-- Loading the prediction data to it
insert into forecasts
select 
  ds,
  link_id,
  y as speed,
  yhat as speed_predicted,
  yhat_upper as speed_predicted_upper,
  yhat_lower as speed_predicted_lower,
  training_date
from link_forecasts;

num_affected_rows,num_inserted_rows
2535280,2535280


In [0]:
results.printSchema()

root
-- ds: date (nullable = true)
-- link_id: string (nullable = true)
-- y: float (nullable = true)
-- yhat: float (nullable = true)
-- yhat_upper: float (nullable = true)
-- yhat_lower: float (nullable = true)
-- training_date: date (nullable = false)

INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0

## Time-Series Analysis Results for individual link_id

In [0]:
%sql

SELECT DISTINCT link_id FROM link_forecasts;

link_id
4616324
4616346
4456510
4616337
4456501
4616318
4763655
4616200
4616218
4616232


**---------- Insert link-Id below to view the predictions. ----------**

In [0]:
%sql
SELECT * FROM link_forecasts
WHERE link_id = '4616192';

ds,link_id,y,yhat,yhat_upper,yhat_lower,training_date
2017-05-24,4616192,49.09,47.952213,62.51553,34.004345,2022-04-24
2017-11-07,4616192,35.41,41.692104,55.210163,27.164154,2022-04-24
2017-11-07,4616192,52.19,46.545486,59.820415,32.34341,2022-04-24
2017-11-08,4616192,39.14,43.991707,57.466694,30.714376,2022-04-24
2017-11-10,4616192,52.81,52.006927,65.83374,37.30972,2022-04-24
2017-11-14,4616192,52.81,51.092117,64.81968,35.851067,2022-04-24
2017-11-15,4616192,50.95,43.27437,56.9359,28.553305,2022-04-24
2017-11-23,4616192,47.84,45.15487,59.163403,30.530085,2022-04-24
2017-11-26,4616192,3.1,42.30861,55.922813,28.568178,2022-04-24
2017-11-30,4616192,26.71,45.637802,59.66028,30.666037,2022-04-24
